In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
# from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *

In [2]:
#function to create api connection to google sheets
def connect_to_gs(service_account_key):
    scopes = ['https://www.googleapis.com/auth/spreadsheets']
    credentials = service_account.Credentials.from_service_account_info(service_account_key, scopes=scopes)
    gs_connection = gspread.authorize(credentials)
    return gs_connection


#create connection
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [3]:
#function to create/update gameweek_results table

In [4]:
#function to create/update player table

In [5]:
#function to create/update drinks table

# def submit_nomination(your_name, nominee, current_week):

#     return None

In [6]:
#function to create/update streaks table

In [7]:
#function to retrieve current game week number

In [16]:
#function to write data to google sheets
def write_google_sheets_data(gc, df, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        df_values = df.values.tolist()
        gs.values_append(sheet_name, {'valueInputOption': 'RAW'}, {
                    'values': df_values})

        return None

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"Error: Worksheet not found, please create a new tab named:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
     

In [17]:
# initialize list of lists
data = [['tom', 10], ['nick', 15], ['juli', 14]]
  
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age'])

df

,Name,Age
0,tom,10
1,nick,15
2,juli,14


In [18]:
write_google_sheets_data(gs_connection, df, "Sheet11", google_sheet_key)

Error: Worksheet not found, please create a new tab named: Sheet11


In [21]:
data = {'Name': [1],
        'Age': [10]}
  
# Create DataFrame
df = pd.DataFrame(data)
df

,Name,Age
0,1,10


In [3]:
#function to fetch data from google sheets
def fetch_google_sheets_data(gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
    
#fetch data


In [119]:
current_week = 1
drinks_df = fetch_google_sheets_data(gs_connection, drinks_table, google_sheet_key,["event"])
drinks_df= drinks_df[drinks_df.event > current_week -3]

In [116]:
latest_drinks = drinks_df.iloc[:,[0,2,3,5,6]]

In [118]:
latest_drinks.rename(columns = {'event':'Game Week', 'drink_name': "Name", "drink_type": "Drink Type", "nomination_deadline_date": "Deadline", "nomination_completed_date":"Completed Date"}, inplace = True)
latest_drinks

/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_34222/930238862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_drinks.rename(columns = {'event':'Game Week', 'drink_name': "Name", "drink_type": "Drink Type", "nomination_deadline_date": "Deadline", "nomination_completed_date":"Completed Date"}, inplace = True)


,Game Week,drinker_name,Drink Type,Deadline,Completed Date
0,1,Connor McDonald,nomination,27/07/23 20:00,Not Completed
1,1,Niklas Wietzorrek,nomination,27/07/23 20:00,Not Completed
2,1,Liam Smorfitt,nomination,27/07/23 20:00,Not Completed


In [84]:
x = ['a', 'b', 'c']
y = x[:]
y.remove('b')

In [87]:
managers_temp = ['Liam Smorfitt', 'Alex Wietzorrek', 'Hethe Brinkman']

In [88]:
red_cards =['Liam Smorfitt', 'Alex Wietzorrek',]

In [9]:
#function to return season metrics
def create_metrics(df):
    
    # Sort the DataFrame in descending order of points, then descending order of total_points then alphabetically
    df_sorted = df.sort_values(['event', 'points', 'total_points', 'player_name'], ascending=[True, False, False, True])

    # Group the DataFrame by event and get the first place finisher for each event
    first_place_df = df_sorted.groupby('event').head(1)

    # Count the occurrences of each entry in the filtered DataFrame
    first_place_player_counts = first_place_df['player_name'].value_counts()

    # Find the player with the most 1st place finishes
    most_1st_place_player = first_place_player_counts.idxmax()
    most_1st_place_count = first_place_player_counts.max()


    # Sort the DataFrame in descending order of points, then descending order of total_points then alphabetically
    df_sorted = df.sort_values(['event', 'points', 'total_points', 'player_name'], ascending=[True, False, False, False])

    # Group the DataFrame by event and get the last place finisher for each event
    last_place_df = df_sorted.groupby('event').tail(1)

    # Count the occurrences of each player in the last place DataFrame
    last_place_player_counts = last_place_df['player_name'].value_counts()

    # Find the player with the most last place finishes
    most_last_place_player = last_place_player_counts.idxmax()
    most_last_place_count = last_place_player_counts.max()


    # Group the DataFrame by player_name and calculate the sum of event_transfer_cost for each player
    player_transfer_cost = df.groupby('player_name')['event_transfers_cost'].sum()

    # Find the player with the highest total event_transfer_cost
    player_with_highest_cost = player_transfer_cost.idxmax()
    player_with_highest_cost_count = player_transfer_cost.max()/4

    # Group the DataFrame by player_name and calculate the sum of points_on_bench for each player
    player_points_on_bench = df.groupby('player_name')['points_on_bench'].sum()

    # Find the player with the highest total points_on_bench
    player_with_highest_points_on_bench = player_points_on_bench.idxmax()
    player_with_highest_points_on_bench_count = player_points_on_bench.max()


    # Find the row with the lowest points across all events
    min_score_row = df.loc[df['points'].idxmin()]

    # Extract the player_name and event for the row with the lowest score
    lowest_score_player_name = min_score_row['player_name']
    lowest_score_event = min_score_row['event']
    lowest_score_points = min_score_row['points']


    return most_1st_place_player, most_1st_place_count, most_last_place_player, most_last_place_count, player_with_highest_cost, player_with_highest_cost_count, player_with_highest_points_on_bench, player_with_highest_points_on_bench_count,lowest_score_player_name,lowest_score_event, lowest_score_points

In [ ]:
#writes nominations to googlesheets

def record_nomination(df, gc, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
    

In [17]:
#fetch nominations data 
df = google_sheets_data(gs_connection, nominations_table, google_sheet_key, [])

,event,nominator_id,nominee_id,nominator_name,nominee_name,nomination_type,nomination_created_date,nomination_deadline_date,nomination_completed_date
0,1,1742534,1947717,Niklas Wietzorrek,Cole Floyd,1,01/01/23,08/01/23,04/01/23
1,2,1930701,1947717,Liam Smorfitt,Cole Floyd,1,10/01/23,17/01/23,13/01/23
2,3,2208836,1947717,Divyam Dixit,Cole Floyd,1,20/01/23,27/01/23,23/01/23
3,4,552345,1947717,Marco Gouveia,Cole Floyd,1,26/01/23,02/02/23,29/01/23
4,4,1947717,1742534,Cole Floyd,Niklas Wietzorrek,2,03/02/23,10/02/23,06/02/23
5,6,552345,1947717,Marco Gouveia,Cole Floyd,1,11/02/23,18/02/23,14/02/23
6,6,1947717,1742534,Cole Floyd,Niklas Wietzorrek,2,19/02/23,26/02/23,22/02/23
7,7,552345,1742534,Marco Gouveia,Niklas Wietzorrek,1,27/02/23,06/03/23,02/03/23
8,7,1742534,534613,Niklas Wietzorrek,Connor McDonald,2,07/03/23,14/03/23,
9,8,1789073,3648627,Devon Jansen,Jason Perry,1,14/03/23,21/03/23,17/03/23


In [26]:
from datetime import datetime

def count_completed_before_deadline(data):
    """
    Count the number of times a nominee completed their nomination before the deadline.

    Parameters:
    data (dict): A dictionary containing the dataset with nominee information.

    Returns:
    int: The count of nominees who completed their nomination before the deadline.
    """
    completed_before_deadline_count = 0

    for i in range(len(data['event'])):
        # Parse date strings into datetime objects
        completion_date = datetime.strptime(data['nomination_completed_date'][i], '%d/%m/%y')
        deadline_date = datetime.strptime(data['nomination_deadline_date'][i], '%d/%m/%y')

        # Check if completion_date is before the deadline_date
        if completion_date < deadline_date:
            completed_before_deadline_count += 1

    return completed_before_deadline_count

In [28]:
df

,event,nominator_id,nominee_id,nominator_name,nominee_name,nomination_type,nomination_created_date,nomination_deadline_date,nomination_completed_date
0,1,1742534,1947717,Niklas Wietzorrek,Cole Floyd,1,01/01/23,08/01/23,04/01/23
1,2,1930701,1947717,Liam Smorfitt,Cole Floyd,1,10/01/23,17/01/23,13/01/23
2,3,2208836,1947717,Divyam Dixit,Cole Floyd,1,20/01/23,27/01/23,23/01/23
3,4,552345,1947717,Marco Gouveia,Cole Floyd,1,26/01/23,02/02/23,29/01/23
4,4,1947717,1742534,Cole Floyd,Niklas Wietzorrek,2,03/02/23,10/02/23,06/02/23
5,6,552345,1947717,Marco Gouveia,Cole Floyd,1,11/02/23,18/02/23,14/02/23
6,6,1947717,1742534,Cole Floyd,Niklas Wietzorrek,2,19/02/23,26/02/23,22/02/23
7,7,552345,1742534,Marco Gouveia,Niklas Wietzorrek,1,27/02/23,06/03/23,02/03/23
8,7,1742534,534613,Niklas Wietzorrek,Connor McDonald,2,07/03/23,14/03/23,
9,8,1789073,3648627,Devon Jansen,Jason Perry,1,14/03/23,21/03/23,17/03/23


In [29]:
count_completed_before_deadline(df.head(6))

6